首先做一个简单的LLM Chain，它只依赖于Prompt给出的信息。
然后，我们做一个retrival chain，它会从数据库中抓取数据，并且将这些数据传递到prompt模版中。
其次，我们还会让LLM记住聊天记录。
最后，我们会做一个agent，它会利用一个LLM去决定是否去抓取数据。

In [5]:
from langchain_openai import ChatOpenAI

In [6]:
llm = ChatOpenAI()

In [7]:
# 向模型提问
llm.invoke("how can langsmith help with testing?")

In [8]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are world class technical documentation writer."),
    ("user", "{input}")
])

In [9]:
chain = prompt | llm  # combine these into a simple LLM chain

In [10]:
chain.invoke({"input":"how can langsmith help with testing?"})

In [11]:
from langchain_core.output_parsers import StrOutputParser
output_parse = StrOutputParser()

In [12]:
chain = prompt | llm | output_parse

In [13]:
chain.invoke({"input":"how can langsmith help with testing?"})

## Hello World for LLM Chain
最简单的langchain应用只包括三个部分，分别是提示词模版、模型和解析模版，分别对应于输入、处理和输出。
提示词模版使用ChatPromptTemplate定义，模型使用ChagOpenAI定义，输出解析器使用StrOutputParser定义

In [14]:
from langchain_core.prompts import ChatPromptTemplate  # 输入
from langchain_openai import ChatOpenAI # 模型处理
from langchain_core.output_parsers import StrOutputParser # 输出

In [15]:
llm = ChatOpenAI()  # 创建模型
prompt = ChatPromptTemplate.from_messages([  # 提示词模版
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])
parser = StrOutputParser() # 将llm的输出解析成字符串

In [16]:
chain = prompt | llm | parser

In [17]:
chain.invoke({"input":"how can langsmith help with testing?"})

## Retrieval Chain
Retrieval Chain的使用可以分为下面的步骤：
1. 获取数据：WebBaseLoader
2. 加载数据：loader.load()
3. 变换：比如文本分段，RecursiveCharacterTextSplitter
4. 生成词向量：embeddings = OpenAIEmbeddings()
5. 存储向量：vectors = FAISS.from_documents(documents, embeddings)
6. 取回：retriever = vectors.as_retriever()

In [18]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain

In [19]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vectors = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectors.as_retriever()

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
<context>
{context}
</context>

Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)  # 创建一个chain，这个chain可以接收一个document
response = document_chain.invoke({
    "input":"how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can help you visualize test result.")]  # 手动传入一个Document
})
print(response)

retriever_chain = create_retrieval_chain(retriever, document_chain)
response = retriever_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

## Conversation Retrieval Chain
<img src="/Users/orange/Downloads/IMG_16E6BAB4A94C-1.jpeg" alt="IMG_16E6BAB4A94C-1" style="zoom:5%;" />

In [20]:
from langchain.chains import create_history_aware_retriever
# MessagePlaceholder类似于一个占位符。
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [21]:
retriever_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the uer's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
retriever_chain = create_history_aware_retriever(llm, retriever, retriever_prompt)
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [22]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM app LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

## Agent

In [23]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information abot LangSmith. For any questions about LangSmith, you must use this tool!"
)

In [39]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = 0
search = TavilySearchResults(tavily_api_key="tavily-hO0quzeD4z8b0sQ0AkjSIF1jkflk7zfn")

In [34]:
tools = [retriever_tool]

In [35]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [36]:
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [37]:
agent_executor.invoke({"input":"what is the weather in SF?"})

## llm和chat_model的区别
llm和chat_model都是表示针对一个特定模型的参数封装。llm和chat_model的区别在于，LLM以字符串作为输入和输出，而chat_model以一系列消息(a list of messages)作为输入并以一个消息作为输出。

In [40]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

llm = OpenAI()
chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [42]:
from langchain_core.messages import HumanMessage
text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

In [43]:
llm.invoke(text)  # 输入一个字符串

In [44]:
chat_model.invoke(messages)  # 输入一个消息列表

## Prompt Templates
提示词模版中会有以"{}"包围的空，我们在把提示词模版实例化为提示词时需要通过键值对的方式指定这些空应该填什么内容。
You can "partial" out variables - e.g. you can format only some of the variables at a time. You can compose them together, easily combining different templates into a single prompt.

In [45]:
from langchain.prompts import PromptTemplate

In [47]:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

PromptTemplate可以将字符串格式化成提示词，ChatPromptTemplate可以将一系列消息格式化成提示词。

In [49]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])
chat_prompt.format_messages(input_language="English", output_language="French", text="I love programing")

## Output parsers
output parser可以将模型的输出转换成下游模型的需要的格式。有下面类型的output parsers，包括：
1. 将LLM的输出转换成结构化信息
2. 将ChatMessage转换成一个字符串
3. 将一些工具的输出转换成一个字符串

In [52]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")

## Composing with LCEL
我们可以将上面提到组件串成一条链(chain)

In [53]:
template = "Generate a list of 5 {text}.\n\n{format_instructions}"

chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())
chain = chat_prompt | chat_model | output_parser
chain.invoke({"text":"colors"})